## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime
import numpy as np



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    
                    
                if name in list(check_sheet.Workflow):
                    print(name)
                    dfy = check_sheet[check_sheet['Workflow']== name]
            
                    dfy.reset_index(drop=True)

                    for index,curie_id in enumerate(dfy.Curie):
                        print(index,curie_id)
                        node_num = dfy.iloc[index][3]
                        query_id = curie_id
                        if np.isnan(dfy.iloc[index][2]) == True:
                            len_check = len(child_response['fields']['data']['message']['results'])
                        else:
                            len_check = dfy.iloc[index][2]

                        #print(node_num, query_id, len_check)

                        locs = []
                        for x, val in enumerate(child_response['fields']['data']['message']['results']):
                            #print(val)

                            if x < len_check:

                                if query_id in val['node_bindings'][node_num][0]['id']:
                                    locs.append(x)
                        if not locs:
                            check_result = f'False'
                            print(check_result)
                            #pass
                        else:
                            check_result = f'True'
                            print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                        dict3[curie_id] = check_result    
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

In [4]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [5]:
check_sheet

,Workflow,Curie,N (size of list of results),Query node ID,Include/Exclude,Unnamed: 5,Unnamed: 6
0,A.0_RHOBTB2_direct.json,PUBCHEM.COMPOUND:2950270,10.0,n1,Include,NaN,Note: this example row means that for workflow...
1,B.1a_DILI-three-hop-from-MONDO:0005359_DILI.json,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
2,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
3,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,CHEMBL.COMPOUND:CHEMBL1698267,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable;...
4,B.1b_DILI_three-hop-from-SNOMEDCT:197358007_To...,RXNORM:40068,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
5,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,RXNORM:155067,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
6,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,CHEMBL.COMPOUND:CHEMBL165,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
7,B.1c_DILI_three-hop-from-MESH:D056487_ChronicD...,CHEMBL.COMPOUND:CHEMBL116438,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable
8,B.1d_DILI_three-hop-from-SNOMEDCT:197354009_To...,CHEMBL.COMPOUND:CHEMBL1698267,500.0,n3,Include,NaN,Equivalent identifiers are equally acceptable;...
9,D.1_parkinsons-crohns.json,NCBIGene:120892,NaN,n01,include,NaN,LRRK2


<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [8]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(500)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=8d4cdd25-3397-41fb-8b15-9a8365095e90
Done
Error
ara-robokop Error 0
Done
ara-arax Done 171
Done
kp-genetics Done 0
Error
ara-explanatory Error 0
Done
ara-aragorn Done 209
Done
kp-molecular Done 63
Unknown
ara-ncats Unknown 0
Done
ara-unsecret Done 66
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-textmining Done 0
Done
ara-bte Done 82
Running
kp-icees-dili Running 0
Done
kp-chp Done 30
Done
kp-cam Done 0
Done
ara-improving Done 5
Done
kp-cohd ARS Error 0
Running
kp-icees Running 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=36f793ee-fa73-414f-a10d-d0e112d62bbe
Done
Error
ara-aragorn-exp Error 0
Error
kp-molecular Error 0
Error
ara-bte Error 0
Done
ara-aragorn Done 0
Unknown
kp-genetics Unknown 0
Error
kp-chp Error 0
Error
kp-textm

https://arax.ncats.io/?source=ARS&id=035768ab-db26-4582-b997-7e7248da7cf2
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.1_DILI-three-hop-related-to.json
B.1_DILI-three-hop-related-to
https://arax.ncats.io/?source=ARS&id=8935e434-95dd-43cc-b15e-1f205b27de64
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol.json
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
https://arax.ncats.io/?source=ARS&id=f41ff986-c8a8-47c2-bd7c-dc7a260b9c00
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin.json
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
https://arax.ncats.io/?source=ARS&id=f222755f-082e-4172-9e66-48e77df956ea
Running
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone.json
B.4g_DILI-fourth-one-hop-

kp-chp Done 10
Running
kp-icees Running 0
Done
ara-improving ARS Error 0
Done
ara-explanatory Done 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowC/C.3a_MultSclerosis_related_to_Nimodipine.json
C.3a_MultSclerosis_related_to_Nimodipine
https://arax.ncats.io/?source=ARS&id=0308b146-41b5-486c-86fd-7645abb62aae
Running
Unknown
ara-ncats Unknown 0
Done
ara-bte Done 0
Done
kp-openpredict Done 0
Done
ara-arax Done 1
Done
ara-explanatory Done 0
Done
ara-improving Done 0
Error
ara-aragorn-exp Error 0
Error
ara-robokop Error 0
Error
ara-aragorn Error 0
Done
kp-textmining Done 0
Done
kp-cohd Done 0
Done
kp-cam Done 0
Done
kp-chp Done 0
Running
kp-icees Running 0
Error
ara-unsecret Error 0
Done
kp-icees-dili Done 0
Done
kp-molecular Done 0
Done
kp-genetics Done 0


<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [9]:
dict_workflows

{'A.3_KCNMA1': '8d4cdd25-3397-41fb-8b15-9a8365095e90',
 'A.2a_RHOBTB2_twohop': '36f793ee-fa73-414f-a10d-d0e112d62bbe',
 'A.8_EGFR_simple': 'c89c6836-c03e-42cc-8c6e-fe1ccf1bb62f',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '9937574f-0532-412b-8185-761565c71add',
 'A.0_RHOBTB2_direct': 'ad2cf3dc-e59f-426f-850e-3c7ee3745a09',
 'A.2_RHOBTB2_twohop': 'bed693f9-54b6-4642-9b11-fcb391ad89e8',
 'A.Multiomics_BigGIM_DR_KP_EGFR': 'd34b129e-a969-415b-aca9-73ece5de7373',
 'A.2_RHOBTB2_twohop_constrained': 'f0108eb5-2133-4c3b-a027-a186799842cc',
 'A.9_EGFR_advanced': '77024745-be18-4d4a-b6c7-9262f4938223',
 'A.2a_expanded_RHOBTB2_twohop_constrained': 'c2de7ec5-f320-40ae-bf33-b3bd0fdc5900',
 'A.1_RHOBTB2': '682bffe3-21a9-4fe1-b798-8e11c5ff42a6',
 'A.2a_expanded_RHOBTB2_twohop': '2d9674d9-847a-4d41-a87f-80b04ac9a599',
 'B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': '1535f25f-b8d4-41a4-a76d-096365a96105',
 'B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone': '571cf404-0707-4a06-88ed-2d20

In [67]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1.json 8d4cdd25-3397-41fb-8b15-9a8365095e90
Done
Error
ara-robokop Error 0
Done
ara-arax Done 171
Done
kp-genetics Done 0
Error
ara-explanatory Error 0
Done
ara-aragorn Done 209
Done
kp-molecular Done 63
Unknown
ara-ncats Unknown 0
Done
ara-unsecret Done 66
Error
ara-aragorn-exp Error 0
Done
kp-openpredict Done 0
Done
kp-textmining Done 0
Done
ara-bte Done 82
Error
kp-icees-dili Error 0
Done
kp-chp Done 30
Done
kp-cam Done 0
Done
ara-improving Done 5
Done
kp-cohd ARS Error 0
Error
kp-icees Error 0
A.2a_RHOBTB2_twohop.json 36f793ee-fa73-414f-a10d-d0e112d62bbe
Done
Error
ara-aragorn-exp Error 0
Error
kp-molecular Error 0
Error
ara-bte Error 0
Done
ara-aragorn Done 0
Unknown
kp-genetics Unknown 0
Error
kp-chp Error 0
Error
kp-textmining Error 0
Error
ara-improving Error 0
Error
ara-ncats Error 0
Error
ara-robokop Error 0
Error
kp-cam Error 0
Error
kp-openpredict Error 0
Error
ara-explanatory Error 0
Error
kp-icees Error 0
Done
ara-unsecret Done 0
Done
ara-arax Done 0
Error
kp-ice

ara-explanatory Done 0
Done
kp-chp Done 0
Error
kp-icees Error 0
Error
ara-bte Error 0
Done
ara-improving ARS Error 0
Unknown
ara-ncats Unknown 0
Error
ara-arax Error 0
Error
ara-robokop Error 0
Error
kp-cohd Error 0
Done
kp-openpredict Done 0
Done
kp-cam Done 0
Error
kp-icees-dili Error 0
Unknown
kp-genetics Unknown 0
Error
ara-aragorn Error 0
Error
kp-molecular Error 0
Error
kp-textmining Error 0
Error
ara-aragorn-exp Error 0
B.5_DILI_branched-four-hop.json c7f94c13-cbf9-45a0-aee1-9545a6695097
Done
Error
ara-unsecret Error 0
Error
kp-icees-dili Error 0
Error
ara-bte Error 0
Error
ara-aragorn Error 0
Done
kp-openpredict Done 0
Error
ara-robokop Error 0
Unknown
ara-ncats Unknown 0
Done
ara-explanatory Done 0
Done
kp-chp Done 0
Done
ara-improving ARS Error 0
Done
kp-textmining Done 0
Unknown
kp-genetics Unknown 0
Error
kp-cohd Error 0
Error
kp-icees Error 0
Error
kp-molecular Error 0
Done
kp-cam Done 0
Error
ara-arax Error 0
Error
ara-aragorn-exp Error 0
B.3g_DILI-fourth-one-hop-from-PU

kp-molecular Done 0
D.1_parkinsons-crohns.json 50ba488f-9dc1-48d4-95db-244bb2e44db7
Done
Error
ara-aragorn Error 0
Unknown
ara-arax Unknown 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-openpredict Done 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-cam Done 0
Error
ara-robokop Error 0
Error
ara-unsecret Error 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
curie id: NCBIGene:120892 : INCLUDED at postion N == [0] on n01
1 NCBIGene:11315
curie id: NCBIGene:11315 : INCLUDED at postion N == [2] on n01
2 NCBIGene:110357
False
ara-improving Done 22
Error
kp-cohd Error 0
Error
kp-molecular Error 0
Unknown
ara-ncats Unknown 0
Done
kp-icees-dili ARS Error 0
Error
kp-icees Error 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
False
2 NCBIGene:110357
False
kp-chp Done 0
Done
D.1_parkinsons-crohns.json
0 NCBIGene:120892
False
1 NCBIGene:11315
Fa

In [11]:
workflow_result_messages

{'A.3_KCNMA1': [{'pk_id': 'https://arax.ncats.io/?source=ARS&id=8d4cdd25-3397-41fb-8b15-9a8365095e90',
   'ara-robokop': 'Error: 500',
   'ara-arax': 'Results: 200{}',
   'kp-genetics': 'No Results: 200',
   'ara-explanatory': 'Error: 404',
   'ara-aragorn': 'Results: 200{}',
   'kp-molecular': 'Results: 200{}',
   'ara-ncats': 'Unknown: 503',
   'ara-unsecret': 'Results: 200{}',
   'ara-aragorn-exp': 'Error: 404',
   'kp-openpredict': 'No Results: 200',
   'kp-textmining': 'No Results: 200',
   'ara-bte': 'Results: 200{}',
   'kp-icees-dili': 'Error: 598',
   'kp-chp': 'Results: 200{}',
   'kp-cam': 'No Results: 200',
   'ara-improving': 'Results: 200{}',
   'kp-cohd': 'ARS Error: 200',
   'kp-icees': 'Error: 598'},
  {'ara-arax': {'infores:arax',
    'infores:biothings-multiomics-biggim-drugresponse',
    'infores:connections-hypothesis',
    'infores:molepro',
    'infores:rtx-kg2',
    'infores:translator-biothings-explorer'},
   'ara-aragorn': {'infores:aragorn',
    'infores:arag

### Creating dataframe for workflows with PK

<br>

In [12]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)


for k in sorted(workflow_result_messages):
    print(k)
    
    if (k == 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule') or (k == 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule') or (k == 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule'):
            pass
    else:
    
        col.append(k)
    
    
    
    #count = 0
        for key, value in workflow_result_messages[k][0].items():
        
        
        #count= count+1
            final_dict[key].append(value)

    final_dict = dict(final_dict)
    #print(count)
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [13]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [14]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=ad2cf3dc-...,https://arax.ncats.io/?source=ARS&id=682bffe3-...,https://arax.ncats.io/?source=ARS&id=bed693f9-...,https://arax.ncats.io/?source=ARS&id=f0108eb5-...,https://arax.ncats.io/?source=ARS&id=36f793ee-...,https://arax.ncats.io/?source=ARS&id=2d9674d9-...,https://arax.ncats.io/?source=ARS&id=c2de7ec5-...,https://arax.ncats.io/?source=ARS&id=8d4cdd25-...,https://arax.ncats.io/?source=ARS&id=c89c6836-...,https://arax.ncats.io/?source=ARS&id=77024745-...,...,https://arax.ncats.io/?source=ARS&id=05aa1f8a-...,https://arax.ncats.io/?source=ARS&id=5a4a2229-...,https://arax.ncats.io/?source=ARS&id=c7f94c13-...,https://arax.ncats.io/?source=ARS&id=08e008ff-...,https://arax.ncats.io/?source=ARS&id=0308b146-...,https://arax.ncats.io/?source=ARS&id=50ba488f-...,https://arax.ncats.io/?source=ARS&id=55c5b3c3-...,https://arax.ncats.io/?source=ARS&id=9c3ffefd-...,https://arax.ncats.io/?source=ARS&id=abd13a57-...,https://arax.ncats.io/?source=ARS&id=b72ff7da-...
ara-improving,Results: 200{'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200{},Results: 200{},Results: 200{},Error: 598,No Results: 200,No Results: 200,Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Results: 200{},No Results: 200,No Results: 200,No Results: 200,"Results: 200{'NCBIGene:120892': 'True', 'NCBIG...","Results: 200{'NCBIGene:3988': 'True', 'NCBIGen...",No Results: 200,Results: 200{},No Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn,No Results: 200,Error: 500,Results: 200{},No Results: 200,No Results: 200,Error: 500,No Results: 200,Results: 200{},Results: 200{},No Results: 200,...,Error: 500,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 500,Error: 500,Error: 500,Error: 598
ara-unsecret,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200{},Results: 200{},No Results: 200,...,No Results: 200,Error: 400,Error: 400,Results: 200{},Error: 400,Error: 400,Error: 400,Error: 400,No Results: 200,No Results: 200
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 404,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-openpredict,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
kp-cohd,Error: 400,Error: 400,Error: 400,Error: 400,Error: 598,Error: 400,Error: 400,No Results: 200,Error: 400,Error: 400,...,No Results: 200,Results: 200{},Error: 400,Results: 200{},No Results: 200,Error: 400,Error: 400,No Results: 200,Error: 400,Error: 400
kp-icees-dili,Error: 598,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 598,Error: 598,No

In [16]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [17]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [18]:
final_dict2

defaultdict(dict,
            {'A.0_RHOBTB2_direct': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'}},
             'A.1_RHOBTB2': {'ara-improving': {'infores:improving-agent',
               'infores:lincs',
               'infores:spoke'},
              'ara-arax': {'infores:arax', 'infores:rtx-kg2'},
              'ara-bte': {'infores:biothings-semmeddb-gene',
               'infores:semmeddb',
               'infores:translator-biothings-explorer'}},
             'A.2_RHOBTB2_twohop': {'ara-bte': {'infores:automat-ctd',
               'infores:automat-gtopdb',
               'infores:automat-hetio',
               'infores:automat-pharos',
               'infores:biothings-dgidb',
               'infores:biothings-semmeddb-gene',
               'infores:ctd',
               'infores:dgidb',
               'infores:gtopdb',
               'infores:hetio',
               'infores:lincs',
               'infores:pharos',
  

In [19]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [20]:
final_dictassemble

[['A.0_RHOBTB2_direct', 'ara-improving', 'infores:improving-agent'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:spoke'],
 ['A.0_RHOBTB2_direct', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:improving-agent'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:spoke'],
 ['A.1_RHOBTB2', 'ara-improving', 'infores:lincs'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:semmeddb'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:biothings-semmeddb-gene'],
 ['A.1_RHOBTB2', 'ara-bte', 'infores:translator-biothings-explorer'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:automat-ctd'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:pharos'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:hetio'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:automat-hetio'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:semmeddb'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 'infores:ctd'],
 ['A.2_RHOBTB2_twohop', 'ara-bte', 

In [21]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [22]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [23]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [24]:
df2test = pd.DataFrame(df2test.unstack().T)

In [25]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.4_tryptophan-kynurenine
Values,,,,,,,,,,,,,,,,,,,,,
infores:aragorn,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn-ranker-ara,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:arax,NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,NaN,NaN,NaN,[ara-arax],NaN,[ara-arax],[ara-arax],NaN,NaN,[ara-arax]
infores:automat-biolink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],NaN,NaN,NaN,NaN,NaN
infores:automat-covidkop,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-ctd,NaN,NaN,"[ara-bte, ara-aragorn]",[ara-bte],[ara-bte],NaN,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],NaN,NaN,NaN,NaN,NaN
infores:automat-drug-central,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,...,NaN,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN,NaN,NaN,NaN
infores:automat-gtopdb,NaN,NaN,"[ara-bte, ara-aragorn]",[ara-bte],NaN,NaN,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hetio,NaN,NaN,"[ara-bte, ara-aragorn]",[ara-bte],NaN,NaN,"[ara-aragorn, ara-bte]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN


In [26]:
#df2test.drop([''], axis=0, inplace=True)

In [27]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [28]:
df2test

Workflow,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.4_tryptophan-kynurenine
Values,,,,,,,,,,,,,,,,,,,,,
infores:aragorn,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:aragorn-ranker-ara,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,[ara-aragorn],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:arax,NaN,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,NaN,NaN,NaN,[ara-arax],NaN,[ara-arax],[ara-arax],NaN,NaN,[ara-arax]
infores:automat-biolink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],NaN,NaN,NaN,NaN,NaN
infores:automat-covidkop,NaN,NaN,[ara-aragorn],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-ctd,NaN,NaN,"[ara-bte, ara-aragorn]",[ara-bte],[ara-bte],NaN,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],NaN,NaN,NaN,NaN,NaN
infores:automat-drug-central,NaN,NaN,NaN,NaN,NaN,NaN,[ara-aragorn],[ara-aragorn],NaN,NaN,...,NaN,NaN,NaN,[ara-bte],[ara-bte],NaN,NaN,NaN,NaN,NaN
infores:automat-gtopdb,NaN,NaN,"[ara-bte, ara-aragorn]",[ara-bte],NaN,NaN,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hetio,NaN,NaN,"[ara-bte, ara-aragorn]",[ara-bte],NaN,NaN,"[ara-aragorn, ara-bte]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[ara-bte],NaN,NaN,NaN,NaN,NaN


In [29]:
#df2.replace([], 'None', regex=True,inplace=True)

In [30]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [31]:
df2test = df2test.rename_axis(None)

In [32]:
df2test.columns.name = None

In [33]:
df2test.fillna('', inplace=True)

In [34]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.4_tryptophan-kynurenine
infores:aragorn,,,[ara-aragorn],,,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,
infores:aragorn-ranker-ara,,,[ara-aragorn],,,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,
infores:arax,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,,,,[ara-arax],,[ara-arax],[ara-arax],,,[ara-arax]
infores:automat-biolink,,,,,,,,,,,...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,,
infores:automat-covidkop,,,[ara-aragorn],,,,,,,,...,,,,,,,,,,
infores:automat-ctd,,,"[ara-bte, ara-aragorn]",[ara-bte],[ara-bte],,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,,
infores:automat-drug-central,,,,,,,[ara-aragorn],[ara-aragorn],,,...,,,,[ara-bte],[ara-bte],,,,,
infores:automat-gtopdb,,,"[ara-bte, ara-aragorn]",[ara-bte],,,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],,...,,,,,,,,,,
infores:automat-hetio,,,"[ara-bte, ara-aragorn]",[ara-bte],,,"[ara-aragorn, ara-bte]",,,,...,,,,,[ara-bte],,,,,
infores:automat-hmdb,,,[ara-aragorn],,[ara-arax],[ara-arax],,,,,...,,,[ara-bte],,,,,,,


## Sorting The index to push infores ID's that are compliant to Column L and M in INFORES Catalog


<br>
<br>

In [35]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [36]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]

In [37]:
infores_catalog = infores_catalog[:335]

In [38]:
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

In [39]:
dict_map

{'infores:aragorn': 'ARA',
 'infores:aragorn-ranker-ara': 'Illegal value',
 'infores:arax': 'ARA',
 'infores:automat-biolink': 'KP',
 'infores:automat-covidkop': 'KP',
 'infores:automat-ctd': 'KP',
 'infores:automat-drug-central': 'KP',
 'infores:automat-gtopdb': 'KP',
 'infores:automat-hetio': 'KP',
 'infores:automat-hmdb': 'KP',
 'infores:automat-pharos': 'KP',
 'infores:automat-robokop': 'KP',
 'infores:bgee': 'External Source',
 'infores:bindingdb': 'External Source',
 'infores:biothings-dgidb': 'KP',
 'infores:biothings-multiomics-biggim-drugresponse': 'KP',
 'infores:biothings-multiomics-clinical-risk': 'KP',
 'infores:biothings-semmeddb-gene': 'KP',
 'infores:chembl': 'External Source',
 'infores:civic': 'External Source',
 'infores:cohd': 'KP',
 'infores:connections-hypothesis': 'KP',
 'infores:cord19-scibite': 'External Source',
 'infores:ctd': 'External Source',
 'infores:dgidb': 'External Source',
 'infores:diseases': 'External Source',
 'infores:drug-repurposing-hub': 'Exte

In [40]:
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)

In [41]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.4_tryptophan-kynurenine,Translator_Category_Complaint_to_ColL&M_InforesCatalog
infores:aragorn,,,[ara-aragorn],,,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,ARA
infores:aragorn-ranker-ara,,,[ara-aragorn],,,,[ara-aragorn],[ara-aragorn],,[ara-aragorn],...,,,,,,,,,,Illegal value
infores:arax,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],...,,,[ara-arax],,[ara-arax],[ara-arax],,,[ara-arax],ARA
infores:automat-biolink,,,,,,,,,,,...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,,,KP
infores:automat-covidkop,,,[ara-aragorn],,,,,,,,...,,,,,,,,,,KP
infores:automat-ctd,,,"[ara-bte, ara-aragorn]",[ara-bte],[ara-bte],,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],"[ara-bte, ara-aragorn, ara-arax]",...,[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,,,KP
infores:automat-drug-central,,,,,,,[ara-aragorn],[ara-aragorn],,,...,,,[ara-bte],[ara-bte],,,,,,KP
infores:automat-gtopdb,,,"[ara-bte, ara-aragorn]",[ara-bte],,,"[ara-aragorn, ara-bte]","[ara-bte, ara-aragorn]",[ara-bte],,...,,,,,,,,,,KP
infores:automat-hetio,,,"[ara-bte, ara-aragorn]",[ara-bte],,,"[ara-aragorn, ara-bte]",,,,...,,,,[ara-bte],,,,,,KP
infores:automat-hmdb,,,[ara-aragorn],,[ara-arax],[ara-arax],,,,,...,,[ara-bte],,,,,,,,KP


In [42]:
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)

In [43]:
df2test

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,...,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.4_tryptophan-kynurenine,Translator_Category_Complaint_to_ColL&M_InforesCatalog,sort
infores:spoke,[ara-improving],[ara-improving],"[ara-aragorn, ara-improving]",[ara-improving],,,[ara-improving],[ara-improving],[ara-improving],,...,,[ara-improving],[ara-improving],,,[ara-improving],[ara-improving],[ara-improving],KP,KP
infores:connections-hypothesis,,,"[ara-arax, ara-aragorn]",[ara-arax],,,"[ara-arax, ara-aragorn, kp-chp]",,,,...,,,,"[ara-arax, kp-chp]",,,,,KP,KP
infores:cohd,,,,,,,,,,,...,,,"[ara-bte, kp-cohd]","[ara-arax, kp-cohd]",,,,,KP,KP
infores:mydisease-info,,,,,,,,,,,...,[ara-bte],[ara-bte],[ara-bte],,,,,,KP,KP
infores:rtx-kg2,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],,...,,[ara-arax],,,[ara-arax],,,[ara-arax],KP,KP
infores:biothings-semmeddb-gene,,[ara-bte],[ara-bte],[ara-bte],[ara-bte],,,,,,...,,,,,,,,,KP,KP
infores:biothings-multiomics-clinical-risk,,,,,,,,,,,...,,,,[ara-bte],,,,,KP,KP
infores:biothings-multiomics-biggim-drugresponse,,,,,,,[ara-arax],,,[ara-arax],...,,,,,,,,,KP,KP
infores:biothings-dgidb,,,[ara-bte],[ara-bte],,,[ara-bte],,,,...,,,,,,,,,KP,KP
infores:lincs,[ara-improving],[ara-improving],"[ara-bte, ara-aragorn, ara-improving]","[ara-bte, ara-improving]",,,[ara-improving],[ara-improving],[ara-improving],[ara-aragorn],...,,,,,,,[ara-improving],,KP,KP


In [44]:
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]

In [45]:
df2test['Query Type'] = 'Sync'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

<br>

## Converting the Pk's to hyperlink

<br>

In [46]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [47]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [48]:
df.sort_index(inplace=True)

In [49]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,Results: 200,Results: 200,No Results: 200,...,Error: 500,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 500,Error: 500,Error: 500,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 598,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,...,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Unknown: 503,Error: 598,Results: 200,Results: 200,Unknown: 503,Error: 598,Unknown: 503,Results: 200,Error: 598
ara-bte,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,Error: 598,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 500
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 404,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200{'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,"Results: 200{'NCBIGene:120892': 'True', 'NCBIG...","Results: 200{'NCBIGene:3988': 'True', 'NCBIGen...",No Results: 200,Results: 200,No Results: 200
ara-ncats,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Error: 598,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
ara-robokop,Error: 500,Error: 500,Error: 500,Error: 500,Error: 598,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,...,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500
ara-unsecret,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,...,No Results: 200,Error: 400,Error: 400,Results: 200,Error: 400,Error: 400,Error: 400,Error: 400,No Results: 200,No Results: 200
kp-cam,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200


<br>

### Highlight the cells In Excel

<br>

In [50]:
def highlight(v):
    if v.startswith('Results'):
        return 'background-color: %s' % 'green'
    elif v.startswith('Error'):
        return 'background-color: %s' % 'red'
    elif v.startswith('No Results'):
        return 'background-color: %s' % 'yellow'
    elif v.startswith('ARS Error'):
        return 'background-color: %s' % 'blue'
    elif v.startswith('Unknown'):
        return 'background-color: %s' % 'magneta'
    else:
        return

In [51]:
styled = df.style.applymap(highlight)

In [52]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-related-to,B.2_DILI-three-hop-interacts-with,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 422,Error: 500,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 500,Error: 500,Error: 500,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 598,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Unknown: 503,Error: 598,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Results: 200,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Error: 400,Results: 200,Unknown: 503,Error: 598,Results: 200,Results: 200,Unknown: 503,Error: 598,Unknown: 503,Results: 200,Error: 598
ara-bte,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 500
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 404,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200{'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Resul

In [53]:
date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
wks_name = 'Workflow Progress Tracker_' + date

In [54]:
wks_name

'Workflow Progress Tracker_2021_09_28-08_13_54_AM'

In [55]:
styled.to_excel('/Users/priyash/Documents/GitHub/minihackathons/Notebooks/' + wks_name + '.xlsx')

#styled.to_excel(writer, sheet_name='progress_tracker')
#df2.to_excel(writer, sheet_name='edge_attribute')

/usr/local/lib/python3.9/site-packages/pandas/io/formats/excel.py:375: CSSWarning: Unhandled color format: 'magneta'
  warnings.warn(f"Unhandled color format: {repr(val)}", CSSWarning)


<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

In [56]:
styled

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_EGFR,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.1_DILI-three-hop-related-to,B.2_DILI-three-hop-interacts-with,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,No Results: 200,Error: 500,Results: 200,No Results: 200,No Results: 200,Error: 500,No Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Error: 500,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 500,Error: 500,Error: 500,Error: 500,Error: 500,Error: 422,Error: 500,Error: 500,Error: 500,Error: 598,Error: 598,Error: 598,Error: 500,Error: 500,Error: 500,Error: 598
ara-aragorn-exp,Error: 404,Error: 404,Error: 404,Error: 404,Error: 598,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404,Error: 404
ara-arax,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Results: 200,Unknown: 503,Error: 598,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Results: 200,Results: 200,Unknown: 503,Unknown: 503,Unknown: 503,Error: 400,Results: 200,Unknown: 503,Error: 598,Results: 200,Results: 200,Unknown: 503,Error: 598,Unknown: 503,Results: 200,Error: 598
ara-bte,Error: 598,Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,Error: 598,Error: 598,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 598,Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,Error: 598,Results: 200,No Results: 200,Error: 598,Error: 598,No Results: 200,No Results: 200,Error: 500
ara-explanatory,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 598,No Results: 200,No Results: 200,Error: 404,No Results: 200,No Results: 200,No Results: 200,Results: 200,No Results: 200,No Results: 200,Error: 504,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,Error: 400,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-improving,Results: 200{'PUBCHEM.COMPOUND:2950270': 'True'},Results: 200,Results: 200,Results: 200,Error: 598,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,No Resul

In [57]:
df['Query Type'] = 'Sync'

df = df[['Query Type']+ list(df.columns[:-1])]

In [58]:
#df.reset_index(inplace=True)

In [59]:
#df.rename({'index': 'ARA/KP'}, axis=1, inplace=True)

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [60]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *

In [61]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

### Push Dataframe 2

<br>

In [63]:
wks2 = 'edge_attribute_source_' + date

In [ ]:
spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
d2g.upload(df2test, spreadsheet_key, wks2, credentials=credentials, row_names=True)